# Example of estimation of N and segmentation

This notebook illustrates the estimation of $N$ and the segmentation on a particular example.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from gudhi import plot_persistence_diagram

import matplotlib.pyplot as plt

from segmentation.estimation_n import estimate_N, estimate_N_from_tree, estimate_N_with_clustering
from segmentation.homology import signal_to_diagram
from segmentation.plotting_utils import plot_intervals, plot_segmentation
from segmentation.segmentation import segment_signal_with_clustering

from f_forms import get_f
from process_utils import get_distance, get_GP

In [ ]:
np.random.seed(37)

Consider the following $f$.

In [ ]:
f = get_f(1)
t = np.linspace(0., 2.5, 100)
_ = plt.plot(t, f(t))
_ = plt.xlabel(r"$t$"), plt.ylabel(r"$f$")

We generate a random $\gamma$, for $N=13$.

In [ ]:
N = 13
t, gamma, _ = get_distance(N=N)
plt.plot(t, gamma)
_ = plt.xlabel(r"$t$"), plt.ylabel(r"$\gamma$")

We define $S = (f\circ\gamma)(t) + W$.

In [ ]:
W = get_GP(t, 0.1, 0.05, 1)[:, 0]

signal = f(gamma)
S = signal + W
_ = plt.plot(t, signal, '--', alpha=0.5, label=r"$f\circ\gamma$")
_ = plt.plot(t, S, label=r"$S$")
_ = plt.legend()
_ = plt.tight_layout()

Even on the noisy signal, the persistence shows two clusters, that we will try to identify in the estimation.

In [ ]:
dgm = signal_to_diagram(S)
_ = plot_persistence_diagram(dgm)

We set $\tau=0.5$

In [ ]:
tau = 0.5
N_hat = estimate_N(dgm, tau)
print(f"The estimated N is {N_hat}.")

We test the estimator introduced in section 6.1 on this example and we plot the function $h_S$.

In [ ]:
N_hat, N_intervals = estimate_N_from_tree(dgm, return_intervals=True)
_ = plot_intervals(N_intervals)
_ = plt.title(r"$\tau\mapsto h_S(\tau)$")
_ = plt.ylabel(r"$\hat{N}_c(S,\tau)$")
_ = plt.xlabel(r"$\tau$")
print(f"The estimated N is {N_hat}.")

We pick $\tau=0.3$ and we verify that the estimation with clustering leads to the same estimation of N.

In [ ]:
tau = 0.35
estimate_N_with_clustering(dgm, tau)

Finally, we segment the signal.

In [ ]:
segmentation = segment_signal_with_clustering(S, tau)
plot_segmentation(S, segmentation)

If we choose a different scale, for example $\tau=0.5$, we see only a single segmentation, made of the more prominent local minima.

In [ ]:
segmentation = segment_signal_with_clustering(S, 0.5)
plot_segmentation(S, segmentation)